In [62]:
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from matplotlib.path import Path
from matplotlib import rc
from scipy.optimize import minimize_scalar, minimize
import datetime
from datetime import timedelta
import time
import os
import networkx as nx
import pandas as pd
import itertools 
from gurobipy import*
from time import process_time
from scipy.optimize import minimize

In [2]:
def sort_vector(v):
    return np.array([v[k] for k in sorted(list(v.keys()))])

def construct_function(n,g):
    ground_set = list(range(1,n+1))
    discrete_concave = sorted(g,reverse =True)
    h = {}
    h[0] = 0
    for i,j in enumerate(discrete_concave):
        h[i+1] = h[i] + j
    return h

def submodular_function(n,g):
    
    func = construct_function(n,g)
    ground = list(range(1,n+1))
    
    def findsubsets(s, l): 
        return list(itertools.combinations(s, l)) 

    subsets = []
    for i in range(1,n+1):
        subsets.extend(findsubsets(ground, i))
        
    f = {}
    f[tuple([0])] = func[0]
    for i in subsets:
        f[i] = func[len(i)]

    return f


def submodular_oracle(S,func,card):
    if card == True:
        return func[len(S)]
    else:
        return func[S]
    
def greedy_submodular(w,func,card):
    
    #find permuation corresponding to cost vector sorted in decreasing order
    pi = np.argsort(-w)+1
    
    #s is the optimal chain of elements in ground set and x is the corresponsing optimal solution constructed by greedy
    x = {}
    s = {}
    s[0] = []
    for i,j in enumerate(pi):
        #extend chain based on permuation above
        s[i+1] = sorted(pi[:i+1])
        
        #x is then the marginal gain
        if w[j-1] > 0:
            x[j] = submodular_oracle(tuple(s[i+1]),func,card) - submodular_oracle(tuple(s[i]),func,card)
        else:
            x[j] = 0
        
    return sort_vector(x)

def greedy_submodular_base(w,func,card):
    
    #find permuation corresponding to cost vector sorted in decreasing order
    pi = np.argsort(-w)+1
    
    #s is the optimal chain of elements in ground set and x is the corresponsing optimal solution constructed by greedy
    x = {}
    s = {}
    s[0] = [0]
    for i,j in enumerate(pi):
        #extend chain based on permuation above
        s[i+1] = sorted(pi[:i+1])
        
        #x is then the marginal gain
        x[j] = submodular_oracle(tuple(s[i+1]),func,card) - submodular_oracle(tuple(s[i]),func,card)
        
    return sort_vector(x)

def greedy_submodular_chains(w,func,chains):
    
    #find permuation corresponding to cost vector sorted in decreasing order
    chains_new = chains + [list(np.ones(len(w)))]
    pi = []
    c_old = np.zeros(len(w))
    pi_prime = np.argsort(-w)
    for c in chains_new:
        c_new = np.array(c) - c_old
        s = list(np.where(c_new == 1)[0])
        for i in pi_prime:
            if i in s:
                pi.append(int(i)+1)
        c_old = c
                
    #s is the optimal chain of elements in ground set and x is the corresponsing optimal solution constructed by greedy
    x = {}
    s = {}
    s[0] = []
    k = int(sum(chains[-1]))
    for i,j in enumerate(pi):
        #extend chain based on permuation above
        s[i+1] = sorted(pi[:i+1])
        
        #x is then the marginal gain
        if w[j-1] >= 0 or i +1 <=k :
            x[j] = submodular_oracle(tuple(s[i+1]),func,card) - submodular_oracle(tuple(s[i]),func,card)
        else:
            x[j] = 0
        
    return sort_vector(x)

In [3]:
def line_search_card(x_0,a,func):
    n = len(x_0)
    card = True

    #check if all components of LS direction are negative since in this case result is trivial
    if all(a <0):
        lam = 0

    #otherwise we run discrete Newtons method for LS    
    else:

        #find initial lam based on singletons in the ground set; also gives check if a is feasible direction to begin with
        lam = min([(submodular_oracle([i+1],func,card) - x_0[i])/j for i,j in enumerate(a) if j > 0])
        
        #we only need to do n iterations since our polytope is cardinality based
        for i in range(len(x_0)):
            #try moving with magnitued lam
            y = x_0 + lam*a

            #sort y so we can check feasiblity in the base polytope
            pi = np.argsort(-y)+1
            
            #find cummulative sums of sorted vector so we can check feasibility/violations
            s = np.cumsum(sorted(y,reverse =True))
            violations = [func[i+1] - j for i,j in enumerate(s)]
            
            #find maximum violation and most violated set
            card_violated,violation_value = min(enumerate(violations), key=lambda x : x[1])
            #card_violated = min([i+1 for i,j in enumerate(violations) if j == violation_value])
            
            
            #if smallest violation is 0 (note they can't all be strictly greater than 0  because of base polytope)
            if np.round(violation_value,4) == 0:
                break
                
            #otherwise we update step size and repeat
            else:
                most_violated_set = pi[:card_violated+1]
                lam = (submodular_oracle(most_violated_set,func,card) - sum([x_0[i-1] for i in most_violated_set]))\
                        /sum([a[i-1] for i in most_violated_set])
                
    #note that if the sum of a != 0, then a will not satisfy base constraint, i.e. its not a feasbile direction
    #So if a is a feasbile direction for the polymatroid but not for the base polytope ground set V will
    #be the most violated in one of the iterations above. Thus, since x_0 \in B(f) we know that x(V) = f(V) 
    # and hence when updating lam the numerator will be 0 for the iteration which would then give that lam = 0
    # this shows the correctness of the algorithm and our construction of the violation for base constraint
    return lam

In [4]:
def get_indicator(S,n):
    dic = {k:0 for k in range(1,n+1)}
    for i in S:
        dic[i] = 1
    return sort_vector(dic)

def get_tight_sets(x,func):
    tight_sets = []
    s = np.cumsum(sorted(x,reverse =True))
    pi = np.argsort(-x)+1
    slack = [func[i+1] - j for i,j in enumerate(s)]
    for i,j in enumerate(slack):
        if np.round(j,4) > 0:
            continue
        else:
            tight_sets.append(get_indicator(pi[:i+1],n))
            
    return tight_sets

In [6]:
#line-search using golden-section rule
def segment_search(f, grad_f, x, y, tol=1e-6, stepsize=True):
    
    '''
    Minimizes f over [x, y], i.e., f(x+gamma*(y-x)) as a function of scalar gamma in [0,1]
    '''
    
    # restrict segment of search to [x, y]
    d = (y-x).copy()
    left, right = x.copy(), y.copy()
    
    # if the minimum is at an endpoint
    if np.dot(d, grad_f(x))*np.dot(d, grad_f(y)) >= 0:
        if f(y) <= f(x):
            return y, 1
        else:
            return x, 0
    
    # apply golden-section method to segment
    gold = (1+np.sqrt(5))/2
    improv = np.inf
    while improv > tol:
        old_left, old_right = left, right
        new = left+(right-left)/(1+gold)
        probe = new+(right-new)/2
        if f(probe) <= f(new):
            left, right = new, right
        else:
            left, right = left, probe
        improv = np.linalg.norm(f(right)-f(old_right))+np.linalg.norm(f(left)-f(old_left))
    x_min = (left+right)/2
    
    # compute step size gamma
    gamma = 0
    if stepsize == True:
        for i in range(len(d)):
            if d[i] != 0:
                gamma = (x_min[i]-x[i])/d[i]
                break
                
    return x_min, gamma


#Fucntion to compute away vertex
def away_step(grad, S):
    costs = {}
    
    for k,v in S.items():
        cost = np.dot(k,grad)
        costs[cost] = [k,v]
    vertex, alpha = costs[max(costs.keys())]  
    return vertex,alpha

#Function to update active set
def update_S(S,gamma, Away, vertex):
    
    S = S.copy()
    vertex = tuple(vertex)
    
    if not Away:
        if vertex not in S.keys():
            S[vertex] = gamma
        else:
            S[vertex] *= (1-gamma)
            S[vertex] += gamma
            
        for k in S.keys():
            if k != vertex:
                S[k] *= (1-gamma)
    else:
        for k in S.keys():
            if k != vertex:
                S[k] *= (1+gamma)
            else:
                S[k] *= (1+gamma)
                S[k] -= gamma
    return {k:v for k,v in S.items() if np.round(v,6) > 0}


def line_search(x, d, gamma_max,func):

    def fun(gamma):
        ls = x + gamma*d
        return func(ls)

    res = minimize_scalar(fun, bounds=(0, gamma_max), method='bounded')

    gamma = res.x
    ls = x + gamma*d        
    return ls, gamma


#AFW Algorithm
def AFW(x, S, lmo, epsilon,func,grad_f, f_tol, time_tol):
    
    #record primal gap, function value, and time every iteration
    now=datetime.datetime.now()
    primal_gap = []
    function_value=[func(x)]
    time = [0]
    f_improv = np.inf

    #initialize starting point and active set
    t = 0    

    while f_improv > f_tol and time[-1] < time_tol:
        
        start = process_time()
        
        #compute gradient
        grad = grad_f(x)

        #solve linear subproblem and compute FW direction
        v = lmo(grad)
        d_FW = v-x

        #If primal gap is small enough - terminate
        if np.dot(-grad,d_FW) <= epsilon:
            break
        else:
            #update convergence data
            primal_gap.append(np.dot(-grad,d_FW))

        #Compute away vertex and direction
        a,alpha_a = away_step(grad, S)
        d_A = x - a

        #Check if FW gap is greater than away gap
        if np.dot(-grad,d_FW) >= np.dot(-grad,d_A):
            #choose FW direction
            d = d_FW
            vertex = v
            gamma_max = 1
            Away = False
        else:
            #choose Away direction
            d = d_A
            vertex = a
            gamma_max = alpha_a/(1-alpha_a)
            Away = True

        #Update next iterate by doing a feasible line-search
        x, gamma = line_search(x, d, gamma_max,func)
        #x, gamma = segment_search(func, grad_f, x, x + gamma_max *d)

        #update active set
        S = update_S(S,gamma, Away, vertex)
        
        end = process_time()
        time.append(time[t] + end - start)
        
        f_improv = function_value[-1] - func(x)
        function_value.append(func(x))
        
        t+=1
        
    return x, function_value, time,t,primal_gap,S

In [256]:
def convex_hull_correction1(S, func):    

    M = np.array([np.array(i) for i in S])
    
    def fun(theta):
        return func(np.dot(M.T,theta))

    cons = ({'type': 'eq', 'fun': lambda theta: sum(theta) - 1}) #sum of theta = 1
    bnds = tuple([(0, 1) for _ in M])
    x0 = tuple([1/len(M) for _ in M])

    res = minimize(fun, x0, bounds=bnds, constraints=cons)
    
    final_S = {tuple(M[i]):res.x[i] for i in range(len(M)) if np.round(res.x[i],5) > 0}

    return np.dot(M.T,res.x),final_S


def convex_hull_correction2(S,q):    

    M = np.array([np.array(i) for i in S])
   
    opt, theta = projection_oracle_vertices(M,q)
    
    final_S = {tuple(M[i]):theta[i] for i in range(len(M))if np.round(theta[i],5) > 0}

    return opt, final_S


def proj_oracle(vertices,y):
    m = Model("opt")
    n = len(vertices[0])
    
    #define variables
    lam = {}
    for i in range(len(vertices)):
        lam[i] = m.addVar(lb=0, name='lam_{}'.format(i))
    
    x = []
    for i in range(n):
        x.append(m.addVar(lb=-GRB.INFINITY, name='x_{}'.format(i)))
    x = np.array(x)
    m.update()

    objExp = 0.5* np.dot(x-y, x-y)
    m.setObjective(objExp, GRB.MINIMIZE)
    m.update()

    #feasibility constraints
    for i in range(n):
        m.addConstr(x[i],'=', sum([lam[j]*vertices[j][i] for j in range(len(vertices))]))

    #convex hull constraint
    m.addConstr(quicksum([lam[i] for i in lam.keys()]), '=',1)
    m.update()
    
    #optimize
    m.setParam( 'OutputFlag', False )
    m.write('exact.lp')
    m.optimize()
    return np.array([i.x for i in x])

In [265]:
def maximal_tight(y,g):
    
    #sort y so we can check feasiblity in the base polytope
    pi = np.argsort(-y)+1

    #find cummulative sums of sorted vector so we can check feasibility/violations
    s = np.cumsum(sorted(y,reverse =True))
    violations = np.round(np.array([g[i+1] - j for i,j in enumerate(s)]),6)
    
    if any (violations < 0):
        return 'y not feasible'
    elif all(violations > 0):
        return [0]
    else:
        return pi[:np.arange(len(y))[violations==0][-1]+1]
    
def proj(x):
    return 0.5*np.dot(x - y,x-y)
    
def proj_grad(x):
    return x - y

def chi(M):
    chi_0 = np.zeros(n)
    for i in M:
        chi_0[i-1] = 1
    return chi_0

def f(s):
    return sum([n + 1 - i for i in range(1,s+1)])

def compute_M(N,x):
    return N[np.round(proj_grad(x)[N - 1],5) == np.round(np.min(proj_grad(x)[N - 1]),5)]

In [266]:
def linear_oracle(A1,A2,b1,b2, c):
    
    '''
    General form LP solver that solves min c^Tx subject to Ax <= b
    '''

    m = Model("opt")
    n = len(A.T)

    #define variables
    x = []
    for i in range(n):
        x.append(m.addVar(lb=-GRB.INFINITY, name='x_{}'.format(i)))

    m.update()              

    objExp = np.dot(np.array(x),c)
    m.setObjective(objExp, GRB.MAXIMIZE)
    m.update()

    #feasibility constraints
    for i in range(len(A1)):
        m.addConstr(np.dot(np.array(x),A1[i]),'<=', b1[i])
        
    for i in range(len(A2)):
        m.addConstr(np.dot(np.array(x),A2[i]),'==', b2[i])


    m.update()

    #convex hull constraint

    m.update()

    #optimize
    m.setParam( 'OutputFlag', False )
    m.optimize()

    return np.array([i.x for i in x]), m.Pi

In [71]:
def online_FW(x, lmo, T, grad_f, G = None):
    
    #record primal gap, function value, and time every iteration
    time = [0]
    grad_list = [np.zeros(len(x))]

    #initialize starting point and active set
    t = 1
    x_t= [x]
    v_t = {}
    loss = []
    
    #define blocksizes and random sampling parameters
    k = int(np.ceil(T**(1/3)))
    
    if G:
        delta  = 2/(G*len(x)**0.5 * k**2)
    else:
        delta  = 2/(len(x)**1.5 * k**2)

    while t <= T:
        
        start = datetime.datetime.now()
        
        if t%k != 0:
            
            #play x_{t-1}, i.e. dont do anything
            x_t.append(x_t[-1])
            
            #observe gradient/loss
            grad = grad_f(x_t[-1])
            grad_list.append(grad)
            
            #compute loss
            loss.append(np.dot(x_t[-1],grad))
            
        else:
            v = []
            for i in range(k):
                v_ = np.random.normal(0,1,n)
                v.append(v_/np.linalg.norm(v_))
            v = np.array(v)
            grad_sum = np.sum(np.array(grad_list),axis = 0)
            x = np.array([lmo(grad_sum + v[j]/delta) for j in range(k)])
            
            #play average of x
            x_t.append(np.mean(x,axis = 0))
            
            #observe gradient/loss
            grad = grad_f(x_t[-1])
            grad_list.append(grad)
            
            #compute loss
            loss.append(np.dot(x_t[-1],grad))

        
        end = datetime.datetime.now()
        time.append(time[t-1] + (end - start).total_seconds())
        
        t+=1
        
    return x_t, time, loss

In [72]:
#permutahedron
def f(s):
    return sum([n + 1 - i for i in range(1,s+1)])
g = {i:f(i) for i in range(1,n+1)}
g[0] = 0


#define projection points
q = np.abs(np.random.normal(0,3,n))
    
#define projection function and its gradient
f = lambda x: 0.5*np.dot(x-q,x-q)
grad_f = lambda x: np.power(x- q,1) + np.random.normal(0,1/n**1,n)


#define greedy linear optimization oracle
lmo = lambda w: greedy_submodular_chains(w,g,[np.ones(len(w))])

#initialize algorithm with arbitrary vertex and point to project
w = np.random.choice(range(1,n+1), size=n, replace=False)
card = True
x  = lmo(w)
T = 10

online_FW(x, lmo, T, grad_f, G = None)

([array([1, 3, 2]),
  array([1, 3, 2]),
  array([1, 3, 2]),
  array([1.33333333, 2.        , 2.66666667]),
  array([1.33333333, 2.        , 2.66666667]),
  array([1.33333333, 2.        , 2.66666667]),
  array([1.66666667, 2.        , 2.33333333]),
  array([1.66666667, 2.        , 2.33333333]),
  array([1.66666667, 2.        , 2.33333333]),
  array([2.        , 1.33333333, 2.66666667]),
  array([2.        , 1.33333333, 2.66666667])],
 [0,
  0.0,
  0.0,
  0.000967,
  0.000967,
  0.000967,
  0.000967,
  0.000967,
  0.000967,
  0.002087,
  0.002087],
 [-3.1293528667551067,
  -1.5382371285976213,
  -1.4248182097092061,
  -2.7634166995210063,
  -0.16005419157112577,
  -5.348056956999331,
  -6.3543301842905935,
  -3.0087583524776367,
  -3.5099786250002,
  -1.7686446680665506])